In [17]:
import sys
import time
import requests
from datetime import datetime, timedelta
import json
import fasttext
import re
from bs4 import BeautifulSoup
import lxml
import numpy as np
import os

MAX_RETRIEVED_ELEMENTS = 1000
PUSHSHIFT_ENDPOINT = 'https://api.pushshift.io/reddit/search/'
THRESHOLD = 0.8
PATTERN = re.compile("&gt;.*?(\n|\s\s)")
HTTP_PATTERN = re.compile(r'[\(\[]?https?:\/\/.*?(\s|\Z)[\r\n]*')
EARLIEST_DATE = datetime(2020, 2, 27) 
DIRNAME = 'pol_analysis/'
PRETRAINED_MODEL_PATH = 'lid.176.bin'
model = fasttext.load_model(PRETRAINED_MODEL_PATH)

In [2]:
# HTTP_PATTERN = re.compile(r'[\(\[]?https?:\/\/.*?(\s|\Z)[\r\n]*')

# reddit_url = 'this is a [https://www.reddit.com/r/LocationReddits/wiki/faq/northamerica] and i think https://www.reddit.com/r/LocationReddits/wiki/faq/northameric'

# hp = 'ra bababa bab'

# re.sub(HTTP_PATTERN, "LINK ", reddit_url)
# # re.sub(r"[ab].*?\s", "LINK", hp)

In [3]:
reddit_url = 'https://www.reddit.com/r/LocationReddits/wiki/faq/northamerica'
page = requests.get(reddit_url, headers = {'User-Agent': 'Mozilla/5.0'})
html = BeautifulSoup(page.text, "lxml")

In [4]:
# p = ['Alabama', 'New Jersey', "Maine", "Ba"]
# for s in p:
#     if len(re.findall(r"[M-Z]", s)) > 0:
#         print(s)

In [5]:
states = html.find_all("h5")[13:]
state_sub_map = {}
for state in states:
    if state.text == 'California':
        state_sub_map[state.text] = ['california']
        continue
#     if len(re.findall(r"[N-Z]", state.text)) <= 0:
#         continue
    links = state.findNext().find_all('a')
    links = [link['href'][3:] for link in links]
    state_sub_map[state.text] = links


In [6]:
cali = html.find_all("h6")
for part in cali:
    links = part.findNext().find_all('a')
    links = [link['href'][3:] for link in links]
    state_sub_map['California'].extend(links)
    


In [7]:
def none_or_empty(text):
    return text is None or len(text) == 0 or text == "[removed]" or text == '[deleted]'

def preprocess(text):
    # Need to get rid of &gt for quotes
    line = PATTERN.sub("", text)
    line = HTTP_PATTERN.sub("LINK ", line)
    # Need to get rid of newlines for fasttext to work
    line = re.sub(r"&gt;.*?(\n|\s\s|\Z)", "", line)
    line = re.sub(r'\s\(\s', " ", line)
    line = re.sub(r"\n", " ", line)
    line = re.sub("\su/[a-zA-Z0-9-_]*(\s|\Z)", " USERNAME ", line)
    line = re.sub("\sr/[a-zA-Z0-9-_]*(\s|\Z)", " REDDIT ", line)    
    line = re.sub("\s{2,}", " ", line)
    return line

In [8]:
# query = PUSHSHIFT_ENDPOINT + 'comment/?q=coronacoaster+continues'
# r = requests.get(query)
# x = r.json()['data']
# # ele  = [ele for ele in x if ele['author'] == 'Duese'][0]['body']
# # ele = ele + "[https://www.reddit.com/r/LocationReddits/wiki/faq/northamerica]"
# x

In [9]:
# preprocess(ele)

In [10]:
def valid_text(text):
    if none_or_empty(text):
        return False, ""
    preproc_text = preprocess(text)
    labels, prob = model.predict(preproc_text)
    if labels[0].endswith('en') and prob[0] >= THRESHOLD:
        return True, preproc_text
    return False, ""
    

In [11]:
def get_dates(latest_date):
    # Get all week increments from earliest date until present
    dates = []
    epochs = []
    curr = EARLIEST_DATE
    while curr < latest_date:
        dates.append(curr.strftime("%m/%d/%Y"))
        epochs.append(int(curr.timestamp()))
        curr += timedelta(weeks=1)
    dates.append(latest_date.strftime("%m/%d/%Y"))
    epochs.append(int(latest_date.timestamp()))
    return dates, epochs

In [22]:
def retrieve_subreddit_data(subreddit, retrieval_type, dirname, fields, keywords, dates):
#     print(len(dates))
    today = datetime.utcnow()
    today_timestamp = int((today - datetime(1970, 1, 1)).total_seconds())
    before_date = today_timestamp
    
    fields = ",".join(fields)
    query_words = "|".join(keywords)
    #query_words = f"-({query_words})"
    text_tag = 'body' if retrieval_type == 'comment' else 'selftext'
   
    count = 0
    total_valid = 0
    diachronic_valid = []
    output_filename = 'r.' + subreddit + '.' + retrieval_type + '.json'
    with open(dirname + output_filename, 'w') as fout:
#         for i in range(len(dates) - 1):
#             after = dates[i] #date farther from present
#             before = dates[i+1] #date closer to present
        epoch_valid = 0 # number of data points in the given time period
        done = False

        while not done:
            count += 1
            query = f"{PUSHSHIFT_ENDPOINT}{retrieval_type}/?subreddit={subreddit}&sort=desc&size={MAX_RETRIEVED_ELEMENTS}" +\
            f"&fields={fields}&q={query_words}&before={before_date}&author=!automoderator"
            # &before={before}&after={after}

#                 query = PUSHSHIFT_ENDPOINT + retrieval_type + '/?subreddit=' + subreddit + \
#                 '&sort=desc&size=' + str(MAX_RETRIEVED_ELEMENTS) + '&before=' + str(before) + '&fields=' + fields + \
#                 "&q=" + query_words + 

            print(query, 'request #', count)
            sys.stdout.flush()
            try:
                r = requests.get(query)
            except:
                print('exception thrown...')
                time.sleep(5)
                continue

            if r.status_code != 200:
                print('bad response code:', r.status_code)
                if r.status_code == 429:
                    time.sleep(3)
                    continue
#                 after += 86400  # slight change to the query
                continue  # retry

# record a submission/comment only if it's not empty

            for i, element in enumerate(r.json()['data']):
#                 latest = element['created_utc']
                if element['author'] == 'AutoModerator':
                    continue
                if 'created_utc' in element.keys():
                    before_date = element['created_utc']
#                     print(before_date)
                    if before_date < int(EARLIEST_DATE.timestamp()):
                        done = True
                        break
                try:
                    is_valid, formatted = valid_text(element[text_tag])
                except KeyError:
                    continue
                if is_valid:
                    element[text_tag] = formatted
                    json.dump(element, fout)
                    fout.write('\n')
                    total_valid += 1
                    epoch_valid += 1

            if len(r.json()['data']) < MAX_RETRIEVED_ELEMENTS:  # end of data
                done = True
#             else:
#                 before_date
#                 after = latest
        diachronic_valid.append(epoch_valid)
                
    print("SUBREDDIT: r/{}".format(subreddit))
    print(retrieval_type)
    print(text_tag)
    print(total_valid)
#     print(len(diachronic_valid))
    return total_valid, np.zeros(22)#diachronic_valid

In [23]:
retrieval_types = ['submission', 'comment']
fields = {'submission': ['author', 'id', 'selftext', 'permalink', 'title', 'created_utc'], 
          'comment': ['author', 'body', 'created_utc', 'permalink']}
keywords = ['covid', 'covid-19', 'coronavirus', 'corona', 'covid19', "pandemic", '"the%20virus"']
#optionally: pandemic?
dates, epochs = get_dates(datetime.utcnow())


output_filename = 'overall_stats_baseline.csv'
diachronic_filename = 'diachronic_stats_baseline.csv'
with open(DIRNAME + output_filename, 'a') as fout:
    with open(DIRNAME + diachronic_filename, 'a') as dfout:
        fout.write('Subreddit,Sample Size\n')
        dfout.write('Subreddit, {}'.format(",".join([f"Week {i+1}" for i in range(len(dates)-1)])))
        dfout.write("\n")
        total_valid = 0
        diachronic_valid = np.zeros(len(epochs)-1)
        for sub in ['democrats', 'republican']:
            if sub[-1] == '/':
                sub = sub[:-1]
            for r_type in retrieval_types:
                sub_valid, sub_diachronic_valid = retrieve_subreddit_data(sub, r_type, DIRNAME, fields[r_type], keywords, epochs)
                total_valid += sub_valid
        fout.write(f"{state},{total_valid}\n")


https://api.pushshift.io/reddit/search/submission/?subreddit=democrats&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1590888281&author=!automoderator request # 1
https://api.pushshift.io/reddit/search/submission/?subreddit=democrats&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1583748034&author=!automoderator request # 2
SUBREDDIT: r/democrats
submission
selftext
38
https://api.pushshift.io/reddit/search/comment/?subreddit=democrats&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus"&before=1590888295&author=!automoderator request # 1
https://api.pushshift.io/reddit/search/comment/?subreddit=democrats&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=covid|covid-19|coronavirus|corona|covid19|pande

# ALL STATES

In [40]:
retrieval_types = ['submission', 'comment']
fields = {'submission': ['author', 'id', 'selftext', 'permalink', 'title', 'created_utc'], 
          'comment': ['author', 'body', 'created_utc', 'permalink']}
keywords = ['covid', 'covid-19', 'coronavirus', 'corona', 'covid19', "pandemic", '"the%20virus"']
#optionally: pandemic?
dates, epochs = get_dates(datetime.utcnow())
dates = dates[:12]
epochs = epochs[:12]

output_filename = 'overall_stats_baseline.csv'
diachronic_filename = 'diachronic_stats_baseline.csv'
with open(DIRNAME + output_filename, 'a') as fout:
    with open(DIRNAME + diachronic_filename, 'a') as dfout:
        fout.write('Subreddit,Sample Size\n')
        dfout.write('Subreddit, {}'.format(",".join([f"Week {i+1}" for i in range(len(dates)-1)])))
        dfout.write("\n")
        for state in state_sub_map:
            print(state)
            total_valid = 0
            diachronic_valid = np.zeros(len(epochs)-1)
            for sub in state_sub_map[state]:
                if sub[-1] == '/':
                    sub = sub[:-1]
                for r_type in retrieval_types:
                    sub_valid, sub_diachronic_valid = retrieve_subreddit_data(sub, r_type, DIRNAME, fields[r_type], keywords, epochs)
#                     diachronic_valid += np.array(sub_diachronic_valid)
                    total_valid += sub_valid
            fout.write(f"{state},{total_valid}\n")
#             for date_idx in range(len(dates)-1):
#                 d_vals = ','.join(map(str, map(int, diachronic_valid)))
#                 dfout.write(f"{state},{d_vals}\n")


Alabama
https://api.pushshift.io/reddit/search/submission/?subreddit=alabama&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1590263932&author=!automoderator request # 1
https://api.pushshift.io/reddit/search/submission/?subreddit=alabama&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1583511814&author=!automoderator request # 2
SUBREDDIT: r/alabama
submission
selftext
252
https://api.pushshift.io/reddit/search/comment/?subreddit=alabama&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1590263938&author=!automoderator request # 1
https://api.pushshift.io/reddit/search/comment/?subreddit=alabama&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=-(covid|covid-19|coronavirus|corona|cov

SUBREDDIT: r/auburn
comment
body
4405
https://api.pushshift.io/reddit/search/submission/?subreddit=baldwincountyal&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1590263997&author=!automoderator request # 1
SUBREDDIT: r/baldwincountyal
submission
selftext
6
https://api.pushshift.io/reddit/search/comment/?subreddit=baldwincountyal&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1590263999&author=!automoderator request # 1
SUBREDDIT: r/baldwincountyal
comment
body
16
https://api.pushshift.io/reddit/search/submission/?subreddit=birmingham&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1590263999&author=!automoderator request # 1
https://api.pushshift.io/reddit/search/submission/?subreddit=birming

SUBREDDIT: r/CalhounCounty
submission
selftext
0
https://api.pushshift.io/reddit/search/comment/?subreddit=CalhounCounty&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1590264081&author=!automoderator request # 1
SUBREDDIT: r/CalhounCounty
comment
body
0
https://api.pushshift.io/reddit/search/submission/?subreddit=Cullman&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1590264082&author=!automoderator request # 1
SUBREDDIT: r/Cullman
submission
selftext
4
https://api.pushshift.io/reddit/search/comment/?subreddit=Cullman&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1590264082&author=!automoderator request # 1
SUBREDDIT: r/Cullman
comment
body
32
https://api.pushshift.io/reddit/search/submission/?subreddit

https://api.pushshift.io/reddit/search/submission/?subreddit=huntsvillealabama&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1585203810&author=!automoderator request # 3
SUBREDDIT: r/huntsvillealabama
submission
selftext
809
https://api.pushshift.io/reddit/search/comment/?subreddit=huntsvillealabama&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1590264108&author=!automoderator request # 1
https://api.pushshift.io/reddit/search/comment/?subreddit=huntsvillealabama&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1590077297&author=!automoderator request # 2
https://api.pushshift.io/reddit/search/comment/?subreddit=huntsvillealabama&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=-(covid|covid

https://api.pushshift.io/reddit/search/comment/?subreddit=huntsvillealabama&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1584194901&author=!automoderator request # 32
https://api.pushshift.io/reddit/search/comment/?subreddit=huntsvillealabama&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1584059836&author=!automoderator request # 33
https://api.pushshift.io/reddit/search/comment/?subreddit=huntsvillealabama&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1583851711&author=!automoderator request # 34
https://api.pushshift.io/reddit/search/comment/?subreddit=huntsvillealabama&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=158346

...

https://api.pushshift.io/reddit/search/comment/?subreddit=madisonwi&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1587272303&author=!automoderator request # 22
https://api.pushshift.io/reddit/search/comment/?subreddit=madisonwi&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1587133886&author=!automoderator request # 23
https://api.pushshift.io/reddit/search/comment/?subreddit=madisonwi&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1587044221&author=!automoderator request # 24
https://api.pushshift.io/reddit/search/comment/?subreddit=madisonwi&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1586919549&author=!automoderator reque

SUBREDDIT: r/madisonwi
comment
body
46826
https://api.pushshift.io/reddit/search/submission/?subreddit=Manitowoc&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1590294307&author=!automoderator request # 1
SUBREDDIT: r/Manitowoc
submission
selftext
0
https://api.pushshift.io/reddit/search/comment/?subreddit=Manitowoc&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1590294309&author=!automoderator request # 1
SUBREDDIT: r/Manitowoc
comment
body
4
https://api.pushshift.io/reddit/search/submission/?subreddit=medfordwi&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1590294311&author=!automoderator request # 1
SUBREDDIT: r/medfordwi
submission
selftext
0
https://api.pushshift.io/reddit/search/commen

https://api.pushshift.io/reddit/search/comment/?subreddit=Milwaukee&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1584761742&author=!automoderator request # 22
https://api.pushshift.io/reddit/search/comment/?subreddit=Milwaukee&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1584583258&author=!automoderator request # 23
https://api.pushshift.io/reddit/search/comment/?subreddit=Milwaukee&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1584389669&author=!automoderator request # 24
https://api.pushshift.io/reddit/search/comment/?subreddit=Milwaukee&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1584139664&author=!automoderator reque

SUBREDDIT: r/swwisconsin
comment
body
0
https://api.pushshift.io/reddit/search/submission/?subreddit=SpartaWI&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1590294420&author=!automoderator request # 1
SUBREDDIT: r/SpartaWI
submission
selftext
0
https://api.pushshift.io/reddit/search/comment/?subreddit=SpartaWI&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1590294422&author=!automoderator request # 1
SUBREDDIT: r/SpartaWI
comment
body
0
https://api.pushshift.io/reddit/search/submission/?subreddit=stevenspoint&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1590294423&author=!automoderator request # 1
SUBREDDIT: r/stevenspoint
submission
selftext
4
https://api.pushshift.io/reddit/search/commen

SUBREDDIT: r/casper
submission
selftext
4
https://api.pushshift.io/reddit/search/comment/?subreddit=casper&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1590294479&author=!automoderator request # 1
SUBREDDIT: r/casper
comment
body
32
https://api.pushshift.io/reddit/search/submission/?subreddit=cheyenne&sort=desc&size=1000&fields=author,id,selftext,permalink,title,created_utc&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1590294480&author=!automoderator request # 1
SUBREDDIT: r/cheyenne
submission
selftext
16
https://api.pushshift.io/reddit/search/comment/?subreddit=cheyenne&sort=desc&size=1000&fields=author,body,created_utc,permalink&q=-(covid|covid-19|coronavirus|corona|covid19|pandemic|"the%20virus")&before=1590294482&author=!automoderator request # 1
SUBREDDIT: r/cheyenne
comment
body
176
https://api.pushshift.io/reddit/search/submission/?subreddit=Cody_WY&sort=

In [140]:
all_files = os.listdir(DIRNAME)
with open(DIRNAME + 'diachronic_state_stats.csv', "w") as dia_file:
    weeks = ",".join([f"Week {i+1}" for i in range(len(dates)-1)])
    dia_file.write(f"State, {weeks}\n")
    for state in state_sub_map:
        if state == 'New York':
            break
        subs = state_sub_map[state]
        state_data = []
        state_json = []
        for sub in subs:
            state_data.extend([file for file in all_files if file.startswith(f"r.{sub}")])
        for file_name in state_data:
            with open(DIRNAME + file_name, 'r') as file:
                state_json.extend([json.loads(line) for line in file.readlines() if line != ''])
        state_json.sort()
        for file_name in state_json:
            
            

#         diachronic = np.bincount(np.digitize(state_json, epochs))
#         d_vals = ','.join(map(str, map(int, diachronic[1:])))
#         dia_file.write(f"{state},{d_vals}\n")
        
        if state == 'California':
            json.dumps(state_json)
        


In [21]:
model.predict("Will biden get Bloomberg's votes? And will bernie have a chance if he does?")

(('__label__en',), array([0.86604697]))